In [1]:
import pandas as pd
import glob
pd.options.mode.chained_assignment = None

In [2]:
# PATHS

#Den Bosch flow
path = "../data/waterschap-aa-en-maas_sewage_2019/sewer_data/data_pump/RG8150/RG8150/"
path1 = "../data/waterschap-aa-en-maas_sewage_2019_db_pumps/sewer_data_db/data_wwtp_flow/RG1876_flow/"
path2 = "../data/waterschap-aa-en-maas_sewage_2019_db_pumps/sewer_data_db/data_wwtp_flow/RG1882_flow/"


#Bokhoven level
path3 = "../data/waterschap-aa-en-maas_sewage_2019/sewer_data/data_pump/RG8180_L0/" 
#Bokhoven flow
path4 = "../data/waterschap-aa-en-maas_sewage_2019/sewer_data/data_pump/RG8180_Q0/" 


#Haarsteeg level
path5 = "../data/waterschap-aa-en-maas-sewage_2019/sewer_data/data_pump/rg8170_N99/" 
#Haarsteeg flow
path6 = "../data/waterschap-aa-en-maas-sewage_2019/sewer_data/data_pump/rg8170_99/" 
#path6 = "../data/waterschap-aa-en-maas_sewage_2019/sewer_data/data_pump/rg8170_99/" 


#Helftheuvelweg level column 003 Helftheuvelweg *.csv
path7 = "../data/waterschap-aa-en-maas_sewage_2019_db/sewer_data_db/data_pump_level/" #path 7, 11, 13, 15 are the same
#Helftheuvelweg flow 
path8 = "../data/waterschap-aa-en-maas_sewage_2019_db/sewer_data_db/data_pump_flow/1210FIT301_99/"


#Engelerschans level column “004 Engelerschans” *.csv
path9 = "../data/waterschap-aa-en-maas_sewage_2019_db/sewer_data_db/data_pump_level/"
#Engelerschans flow + Haarsteeg + Bokhoven, therefore substract for only Engeleschans
path10 = "../data/waterschap-aa-en-maas_sewage_2019_db/sewer_data_db/data_pump_flow/1210FIT201_99/"


#Maaspoort level Column: “006 Maaspoort” *.csv 
path11 = "../data/waterschap-aa-en-maas_sewage_2019_db/sewer_data_db/data_pump_level/"
#Maasport flow + Rompert
path12= "../data/waterschap-aa-en-maas_sewage_2019_db/sewer_data_db/data_pump_flow/1210FIT501_99/" #path 12, 16 are the same


#Oude Engelenseweg level Column: “002 Oude Engelenseweg” *.csv
path13 = "../data/waterschap-aa-en-maas_sewage_2019_db/sewer_data_db/data_pump_level/"
#Oude Engelenseweg flow
path14 = "../data/waterschap-aa-en-maas_sewage_2019_db/sewer_data_db/data_pump_flow/1210FIT401_94/"


#De Rompert level Column: “005 de Rompert” *.csv
path15 = "../data/waterschap-aa-en-maas_sewage_2019_db/sewer_data_db/data_pump_level/"
#De Rompert flow + Maasport
path16 = "../data/waterschap-aa-en-maas_sewage_2019_db/sewer_data_db/data_pump_flow/1210FIT501_99/"

In [3]:
#Reading and rendering (One liner example)
#Use for example this flow data and the below rain data for Haarsteeg to experiment with visualizations
#Haarsteeg



#extention = 'csv'

#all_filenames = [i for i in glob.glob('*.{}'.format(extention))]

#combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ], ignore_index=True)

#combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')


#Emilyans code is below

df = pd.concat([pd.read_csv(file) for file in glob.glob(path5+'/*.*')], ignore_index = True)

#df.to_csv(path5+"haarsteeg_l_comb.csv")

In [4]:
#Location linkage
path_linkinfo = "../data/waterschap-aa-en-maas_sewage_2019/sewer_model/"
path_rain = "../data/waterschap-aa-en-maas-sewage_2019/sewer_data/rain_timeseries/"

#Missing Engelerschans and RWZI Den Bosch linking streets inside the linking document(Someone revisit this please e.g @Roger)
#Another missing thing this time in the rainfall data itself are streets in "Hertogenbosch (Oude Engelenseweg)" and in "Hertogenbosch (Maaspoort)", if you want to
#see, plug "Hertogenbosch (Oude Engelenseweg)" or the other one in station_names to see exactly which streets are not in index
# THIS IS SOMETHING TO REPORT AS MISSING IN OUR DELIVERABLE!!!

station_names = ["Haarsteeg", "Bokhoven", "Hertogenbosch (Helftheuvelweg)", "Hertogenbosch (Rompert)"]
link = pd.read_csv("../data/waterschap-aa-en-maas-sewage_2019/sewer_model/20180717_dump riodat rioleringsdeelgebieden_matched_to_rainfall_locations.csv", header=9)

#Street names by stations
streets = [list(link[link["Naam kern"] == name]["Naam / lokatie"]) for name in station_names]

rain = pd.concat([pd.read_csv(file, header = 2) for file in glob.glob(path_rain+"*.*")], ignore_index = True)

#All the rain for the streets for the pump stations in order of station_names and the streets per station
#can be found in streets nested list in the same order


all_areas = rain[["Begin", "Eind"] + [i for sl in streets for i in sl]]


In [5]:
all_areas2 = pd.DataFrame()

for i in range(len(station_names)):
    all_areas2[station_names[i]] = rain[streets[i]].sum(1)

all_areas2["Begin"] =  all_areas["Begin"]
all_areas2["End"] = all_areas["Eind"]

In [6]:
all_areas2["Begin"] = pd.to_datetime(all_areas2["Begin"])
all_areas2["date"] = all_areas2["Begin"].dt.date

# The sum of the rain at all locations per day
all_areas2_days = all_areas2.groupby('date').sum()

In [7]:
def condition_days(rain_value): #conditions are made based on wikipedia numbers, intensity section
    if rain_value == 0:         #https://en.wikipedia.org/wiki/Rain
        return "No"
    elif 0 < rain_value <= 2.352 :
        return 'light'
    elif 2.352 < rain_value <= 7.2:
        return 'moderate'
    elif 7.2 < rain_value <= 48:
        return 'heavy'
    elif 48 < rain_value:
        return 'Violent'
    else:
        return 'Something went wrong'

In [8]:
def level_of_rain(names_list, df):
    for i in names_list:
        df["Rain measure " + i] = df.apply(lambda x: condition_days(x[i]), axis=1)
    return df    

In [9]:
level_of_rain(station_names, all_areas2_days)

,Haarsteeg,Bokhoven,Hertogenbosch (Helftheuvelweg),Hertogenbosch (Rompert),Rain measure Haarsteeg,Rain measure Bokhoven,Rain measure Hertogenbosch (Helftheuvelweg),Rain measure Hertogenbosch (Rompert)
date,,,,,,,,
2017-12-31,0.0000,0.0000,0.0000,0.0000,No,No,No,No
2018-01-01,25.6440,15.7751,82.1657,28.1375,heavy,heavy,Violent,heavy
2018-01-02,14.4315,8.1431,33.4931,11.1320,heavy,heavy,heavy,heavy
2018-01-03,0.0000,0.0000,0.0000,0.0000,No,No,No,No
2018-01-04,4.0660,3.3450,15.2095,4.6060,moderate,moderate,heavy,moderate
2018-01-05,11.3058,7.5352,37.7321,14.0173,heavy,heavy,heavy,heavy
2018-01-06,39.6318,31.1847,139.1714,55.6652,heavy,heavy,Violent,Violent
2018-01-07,0.0000,0.0000,0.0000,0.0000,No,No,No,No
2018-01-08,0.0000,0.0000,0.0000,0.0000,No,No,No,No


In [11]:
all_areas2_days = all_areas2_days.reset_index()

In [16]:
#adding the columns for moderate, heavy rain from the dataset with days to the dataset with timestamps every 5min
#the values there are exactly the same as we do not do different calculations for the bigger dataset, we simply rewrite the
#values for the visualisation purposes
all_areas3 = pd.merge(all_areas2, all_areas2_days, on='date', how='outer')